**NOTE** `phimats` environment should be used as kernel

In [1]:
import numpy as np
import h5py

import PreProcessing
from PreProcessing import PreProcessing as PP, ReadMesh

import BoundaryConditions
from BoundaryConditions import *

import PostProcessing
from PostProcessing import WriteXDMF

### Simulation data

In [ ]:
# Simulation name
Simul = "ChargingHT990"
# Simulation type
SimulType = "2PhaseTrapping" 
# Element sets
nElementSets = 1
# Number of steps
nSteps = 20

### Read mesh file

In [3]:
# Path to .inp mesh file
modelName = "../RVE_HT990"

# Create meshio mesh object
mesh = ReadMesh(modelName)
print(mesh)

<meshio mesh object>
  Number of points: 370990
  Number of cells:
    triangle: 740467
  Point sets: plate
  Cell sets: plate


### Element data

In [4]:
elementName = "triangle" 	# meshio compatible element name
nNodes = mesh.points.shape[0]            # Total number of nodes
nodCoord = mesh.points[:,0:2]            # Node coordinates
nElements = mesh.cells_dict[elementName].data.shape[0]  # Total number of elements
elemNodeConn = mesh.cells_dict[elementName]        # Node connectivity for all elements

### Read phase-field data

In [ ]:
# Path to the phase-field RVE data
fh5 = h5py.File("../RVE_HT990.hdf5", "r")

gPhi_jj = fh5["gPhi_jj"][()]   
gPhi_ij = fh5["gPhi_ij"][()]   
gPhi_ii = fh5["gPhi_ii"][()]   
phi_j   = fh5["phi_j"][()]   

fh5.close()

### Diffusivity parameters

In [ ]:
T = 281                    # Temperature [K] 8C

R = 8.31446261815324       # Universal gas constant [J/mol.K]

dt = 864                   # Time increment [s]

# Lattice diffusivity m²/s
D01 = 8.45e-6; DQ1 = 37700  
DL = D01*np.exp(-DQ1/(R*T)) 

# High solubility phase diffusivity
D02 = 5.76e-7; DQ2 = 49620 
DT = D02*np.exp(-DQ2/(R*T))

print("DL: ", DL)
print("DT: ", DT)
print("DL/DT: ", DL/DT)

DL:  8.298525128712058e-13
DT:  3.4420742950361795e-16
DL/DT:  2410.9081958746133


### Solubility parameters

In [14]:
Ejj       = 2000
Eij       = 5000
Eii       = 10000

Ej        = 12000

zeta_ii = 4*Eii
print("Zii: ", np.exp(zeta_ii/(4*R*T)))

zeta_ij = 4*Eij  # What actually happes is 4*(Eij+Ej) due to diffuse interface
print("Zij: ", np.exp(zeta_ij/(4*R*T)))

zeta_jj = 4*Ejj   # What actually happes is 4*(Ejj+Ej) due to diffuse interface
print("Zjj: ", np.exp(zeta_jj/(4*R*T)))

zeta_j = Ej
print("Zj: ", np.exp(zeta_j/(R*T)))

Zii:  72.25163696760185
Zij:  8.500096291666457
Zjj:  2.3537998900397787
Zj:  170.06589514953538


In [ ]:
# Material `dict`
Materials = {
    "Material_1" : {
		"D0x1" : D01,
		"D0y1" : D01,
		"DQx1" : DQ1,
		"DQy1" : DQ1,
		"D0x2" : D02,
		"D0y2" : D02,
		"DQx2" : DQ2,
		"DQy2" : DQ2,
		"zeta_j"   : zeta_j,
		"zeta_jj"  : zeta_jj,
		"zeta_ij"  : zeta_ij,
		"zeta_ii"  : zeta_ii,
    }
}

### Apply boundary conditions

In [16]:
# Concentration wppm * Fe density kg/m³
mass_h = 5 * 7.87
mol_h = mass_h/1.008  # this is per m³
Con_b = mol_h
print(Con_b)

39.03769841269841


In [18]:
ly = 60e-6    # height of the RVE
lx = 100e-6   # width of the RVE

exitNods = []
conBCs, exitNods = PermeationX(lx, Con_b, mesh, Charging=True)

# Equilibrium values for Dirichlet boundary conditions
for i in range(len(conBCs)):
    conBCs[i][1] = conBCs[i][1]*np.exp((gPhi_ii[conBCs[i][0]]*zeta_ii/(4*R*T)) + 
                                   	   (gPhi_ij[conBCs[i][0]]*zeta_ij/(4*R*T)) +
                                 	   (gPhi_jj[conBCs[i][0]]*zeta_jj/(4*R*T)) +
                                 	   (phi_j[conBCs[i][0]]*zeta_j/(R*T)))

# Write boundary conditions in a .vtu file for checking in Paraview. recommended view is `Points`. 
BCmesh = WriteConBCs(Simul, elementName, mesh, conBCs)

### Initialize simulation object

In [ ]:
# All input data `dict``
SimulationData = {
    "Simul"         : Simul,
    "SimulType"		: SimulType,
    "mesh"          : mesh,
    "gPhi_ii"       : gPhi_ii,
    "gPhi_ij"       : gPhi_ij,
    "gPhi_jj"       : gPhi_jj,
    "phi_j"    		: phi_j,
    "elementName"   : elementName,
    "nElementSets"  : nElementSets,
    "presBCs"       : conBCs,
    "exitNods"      : exitNods,
    "dt"            : dt,
    "nSteps"        : nSteps,
    "Materials"     : Materials,
    "T"             : T,
}

# Preprocessing object
Duplex_Ch = PP(SimulationData)

# Create the input file Simul+"_in.hdf5" file
Duplex_Ch.WriteInputFile()

HDF5 file 'ChargingHT990_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [ ]:
# Create the output file Simul+"_out.hdf5" file
Duplex_Ch.WriteOutputFile()

### Generate visualization xdmf file

In [23]:
WriteXDMF(Simul, elementName, nSteps, "Transport2D", tOut=1)
# Note that `tOut` should be same as that in the .cxx driver file 